In [12]:
import pickle
import os
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score
import librosa
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('commands.csv')
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname, df.label))
p_path = os.path.join('pickles', 'conv.p')

with open(p_path, 'rb') as handle:
	config = pickle.load(handle)

model = load_model(config.model_path)

In [14]:
y_true = []
y_pred = []

In [27]:
print('Extracting features from audio')
rate, wav = wavfile.read('cleanAudio/output.wav')
label = 'abort'
c = classes.index(label)
y_prob = []

Extracting features from audio


In [28]:
for i in range(0, wav.shape[0]-config.step, config.step):
	sample = wav[i:i+config.step]
	x = mfcc(sample, rate, numcep=config.nfeat, nfilt=config.nfilt, nfft=config.nfft)
	x = (x - config.min) / (config.max - config.min)

	if config.mode == 'conv':
		x = x.reshape(1, x.shape[0], x.shape[1], 1)
	elif config.mode == 'time':
		x = np.expand_dims(x, axis=0)
	y_hat = model.predict(x)
	y_prob.append(y_hat)
	y_pred.append(np.argmax(y_hat))
	y_true.append(c)

In [30]:
fn_prob = np.mean(y_prob, axis=0).flatten()
print(classes[np.argmax(fn_prob)])

centroid


In [31]:
fn_prob.sum()

1.0

In [32]:
fn_prob

array([5.3506717e-03, 9.6144408e-02, 7.5989246e-01, 2.1176422e-05,
       2.2329239e-04, 9.1612287e-04, 1.3736312e-01, 8.8729168e-05],
      dtype=float32)